In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-talk')

from tqdm import tqdm

import torch

# Task #2

A template code for training an RBM on Rydberg atom data (the full dataset) is provided below. For the first part of this task (determining the minimum number of hidden units), start with 20 hidden units. 

Imports and loadining in data:

In [4]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from RBM_helper import RBM

import Rydberg_energy_calculator

training_data = torch.from_numpy(np.loadtxt("Rydberg_data.txt"))

In [ ]:
training_data.size()

torch.Size([20000, 100])

Define the RBM:

In [5]:
n_vis = training_data.shape[1]
n_hin = 1

rbm = RBM(n_vis, n_hin)

Train the RBM:

In [ ]:
epochs = 100
num_samples = 200

exact_energy = -4.1203519096
print("Exact energy: ",exact_energy)

for e in range(1, epochs+1):
    # do one epoch of training
    rbm.train(training_data)   
 
    # now generate samples and calculate the energy
    if e % 100 == 0:
        print("\nEpoch: ", e)
        print("Sampling...")

        init_state = torch.zeros(num_samples, n_vis)
        RBM_samples = rbm.draw_samples(100, init_state)

        print("Done sampling. Calculating energy...") 
 
        energies = Rydberg_energy_calculator.energy(RBM_samples, rbm.wavefunction) 
        print("Energy from RBM samples: ", energies.item())

Exact energy:  -4.1203519096

Epoch:  100
Sampling...
Done sampling. Calculating energy...
Energy from RBM samples:  -4.120345998271888


# Part1: How many hidden units?

In [5]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-talk')

import datetime
import pickle

import torch
from RBM_helper import RBM

import Rydberg_energy_calculator

training_data = torch.from_numpy(np.loadtxt("Rydberg_data.txt"))

epochs = 100
num_samples = 200

exact_energy = -4.1203519096

In [6]:
n_vis = training_data.shape[1]
n_hin = 1

rbm = RBM(n_vis, n_hin)



epochs = 100
num_samples = 200

exact_energy = -4.1203519096
print("Exact energy: ",exact_energy)

Exact energy:  -4.1203519096


In [7]:
'''
I am going to transform the code above to a 
function that returns min n_h needed. 
'''

def min_nh_finder(training_data, num_samples = 200, epochs = 100, Trshld = .0001, nh_max = 10, n_test = 5 ):

    n_h_list = np.arange(1,nh_max, 1)#np.array([1, 3, 6, 9 ])  ## List of the values we want to try for the #hidden units. 
    test_rate = int(epochs/n_test)

    Met_Learning_Cond = False
    error_list_list = []

    Min_nh_Needed = 0
    n_vis = training_data.shape[1]


    for n_h in n_h_list:

        error_list = []

        rbm = RBM(n_vis, n_h)
        rbm.initialize_parameters()

        print(f'\n \n************\n Now let\'s try {n_h} hidden units. ')
        for e in range(1, epochs+1):
            # do one epoch of training
            rbm.train(training_data)   

            ## Stop condition
            if e % test_rate == 0:
                print("\n Epoch: ", e)
        #         print("Sampling...")

                init_state = torch.zeros(num_samples, n_vis)
                RBM_samples = rbm.draw_samples(100, init_state)

        #         print("Done sampling. Calculating energy...") 

                energies = Rydberg_energy_calculator.energy(RBM_samples, rbm.wavefunction) 
                print(" Energy from RBM samples: ", energies.item())

                energy_difference = np.abs(energies.item() - exact_energy) ## Note that this has to be positive. 
                print(" The error (energy difference) is : ", energy_difference )
                error_list += [ [e,energy_difference] ]
                
                if np.abs(energy_difference) <Trshld:
#                     print('Yeah, met the learning condition!')
                    Min_nh_Needed = n_h
                    
                    ### This is to save the model. 
                    now = datetime.datetime.now( )
                    model_name = f'Task2_Part1_{now.day}-{now.hour}-{now.minute}.pkl'
                    print(f'The model was saved as {model_name}.pkl. ')
                    with  open(model_name , 'wb') as f:
                        pickle.dump(rbm, f)
                    Met_Learning_Cond = True
                    break
#                 else:
#                     print('Be a bit more patient, I\'m working on it!')

        error_list_list += [error_list]

        ## If satisfied, we need the break the outter loop too. 
        if Met_Learning_Cond:
            break

    if Met_Learning_Cond:
        print('\n========\n********\n Training successful!')
        print(f' Learning criterion reached in {e} epoch for {Min_nh_Needed} hidden nodes. ')
        return Min_nh_Needed, error_list_list, n_h_list
    else:
        print('\n========\n********\n Did not get there! need to try more hidden units...')
        return -1, error_list_list, n_h_list  # This is to indicate that it did not converge. 



In [8]:
nh_min, e_list, nh_list = min_nh_finder(training_data, Trshld=.0001, epochs=500, num_samples=2000)


 
************
 Now let's try 1 hidden units. 

 Epoch:  100


/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


 Energy from RBM samples:  -4.1200389136921745
 The error (energy difference) is :  0.0003129959078256306

 Epoch:  200
 Energy from RBM samples:  -4.1202445638418235
 The error (energy difference) is :  0.00010734575817661351

 Epoch:  300
 Energy from RBM samples:  -4.120122572598137
 The error (energy difference) is :  0.00022933700186289485

 Epoch:  400
 Energy from RBM samples:  -4.119763410523758
 The error (energy difference) is :  0.0005884990762421438

 Epoch:  500
 Energy from RBM samples:  -4.120173920185927
 The error (energy difference) is :  0.00017798941407320967

 
************
 Now let's try 2 hidden units. 

 Epoch:  100
 Energy from RBM samples:  -4.1198197684021505
 The error (energy difference) is :  0.0005321411978496116

 Epoch:  200
 Energy from RBM samples:  -4.11953958173771
 The error (energy difference) is :  0.0008123278622900187

 Epoch:  300
 Energy from RBM samples:  -4.119949186294541
 The error (energy difference) is :  0.0004027233054593182

 Epoch: 

/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [ ]:
with  open('Task2_Part1_nh3.pkl', 'rb') as f:
    model = pickle.load(f)

In [9]:
init_state = torch.zeros(num_samples, n_vis)
RBM_samples = rbm.draw_samples(100, init_state)

print("Done sampling. Calculating energy...") 

energies = Rydberg_energy_calculator.energy(RBM_samples, rbm.wavefunction) 
energies



Done sampling. Calculating energy...


tensor(-1.0371, dtype=torch.float64)

This is not right. There is a problem. This does not reproduce the results. 

## Part 2